**DATASET - selezione e importazione**
---

WASHINGTON
1.  EV - Population (https://catalog.data.gov/dataset/electric-vehicle-population-data)
2. EV - Adoption Rate (https://data.wa.gov/Transportation/Monthly-Electric-Vehicle-Adoption-Rate-by-County/crrp-awfs
)

ALTRI
1.   Connecticut EV - Chiarging Stations (https://catalog.data.gov/dataset/electric-vehicle-charging-stations)
2.   Dataset strano (https://catalog.data.gov/dataset/motor-vehicle-registrations-dashboard-data)


In [ ]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output


pop = pd.read_csv(r"C:\Users\Piate\Desktop\Uni\TERZO ANNO\ARD\DATASET_ESAME\Electric_Vehicle_Population_Data.csv")
adopt = pd.read_csv(r"C:\Users\Piate\Desktop\Uni\TERZO ANNO\ARD\DATASET_ESAME\Electric_Vehicle_Population_Size_History_By_County.csv")

In [ ]:
pop.columns

In [ ]:
adopt.columns


**PULIZIA dei DATASET**
---


> **pop**

1. eliminare righe che non hanno WA, e poi tutta la colonna stato
2. eliminare colonna VIN
3. eliminare Electric Utility
4. eliminare 2020 Censun Tract
5. eliminare Base MRP
6. eliminare Electric Range (utile ai fini dell'analisi dello stato?)
7. Postal Code serve per la mappa?



> **adopt**

1. eliminare colonna stato WA
2. eliminare righe Vehicle Primary Use = "truck"


In [ ]:

#Rimozione colonne futili

pop.drop('VIN (1-10)', axis=1, inplace=True)
pop.drop('Electric Utility', axis=1, inplace=True)
pop.drop('2020 Census Tract', axis=1, inplace=True)
pop.drop('Base MSRP', axis=1, inplace=True)
pop.drop('Electric Range', axis=1, inplace=True)
pop.drop('Clean Alternative Fuel Vehicle (CAFV) Eligibility', axis=1, inplace=True)
pop.drop('Legislative District', axis=1, inplace=True)
pop.drop('DOL Vehicle ID', axis=1, inplace=True)
pop.drop('Vehicle Location', axis=1, inplace=True)

#Rinmina colonne

pop.rename(columns = {'Postal Code':'Postal_Code'}, inplace = True)
pop.rename(columns = {'Model Year':'Model_Year'}, inplace = True)
pop.rename(columns = {'Electric Vehicle Type':'EV_Type'}, inplace = True)
pop.rename(columns = {'Vehicle Location':'Vehicle_Location'}, inplace = True)


pop_tmt = pop[pop.State == "WA"]

pop_tmt

In [ ]:

#Pulizia del data_set Adopt (Solo rinomina colonne)

adopt.rename(columns = {'Vehicle Primary Use':'User'}, inplace = True)
adopt.rename(columns = {'Battery Electric Vehicles (BEVs)':'Battery_EV'}, inplace = True)
adopt.rename(columns = {'Plug-In Hybrid Electric Vehicles (PHEVs)':'Plugin_Hybrid_EV'}, inplace = True)
adopt.rename(columns = {'Electric Vehicle (EV) Total':'EV_Total'}, inplace = True)
adopt.rename(columns = {'Non-Electric Vehicle Total':'Non_EV_Total'}, inplace = True)
adopt.rename(columns = {'Total Vehicles':'Vehicles_Total'}, inplace = True)
adopt.rename(columns = {'Percent Electric Vehicles':'Percent_EV'}, inplace = True)
adopt[adopt.User == "Passenger"]


In [ ]:
# df3 e df3_vecchio ottenuti dopo query SQL e formattazione della data da Excel (il vechio ha 29 Febbraio 2020)
df3_vecchio = pd.read_csv(r"C:\Users\Piate\Desktop\Uni\TERZO ANNO\ARD\DATASET_ESAME\DATASET_ESAME_date_pulite.csv")
df3 = pd.read_csv(r"C:\Users\Piate\Desktop\Uni\TERZO ANNO\ARD\DATASET_ESAME\DATASET_ESAME_date_pulite-_1_.csv")
df4 = pd.read_excel(r"C:\Users\Piate\Desktop\Uni\TERZO ANNO\ARD\DATASET_ESAME\DATASET_ESAME_date_pulite.xlsx")

**PLANNING DEI GRAFICI**
===


> **Grafico 1** - Bar Chart (con confronto totale)

Andamento temporale hybrid vs plug-in (https://data.wa.gov/Transportation/Electric-Vehicle-Population-Counts/qi6z-wzah)



> **Grafico 2** - pie chart

Percentuali modelli/marchio elettrici per anno



> **Grafico 3** - mapbox

mappa per contea per anno








**GRAFICO 1**
---

In [ ]:
#plotto il grafico - tentativo riuscito con id-colore
#possibile soluzione per evitare colonna ID_Colore: {int(i): str(i) for i in df1.Plugin_Hybrid_EV : 'red'} --> problema: da errore

fig0 = px.line(df3, y='EV_Total', x='Date', color = 'ID_Colore_Totale', color_discrete_map= {'Totale' : 'red'})
fig1 = px.bar(df3, y='Plugin_Hybrid_EV', x='Date', color = 'ID_Colore_Plugin', color_discrete_map= {'Ibrido plugin' : '#3366CC'})
fig2 = px.bar(df3, y='Battery_EV', x='Date', color = 'ID_Colore_Batteria', color_discrete_map= {'Batteria' : "#0099c6"})

fig = go.Figure(data = fig1.data + fig2.data + fig0.data)
fig.update_layout(barmode="stack", clickmode="event+select")

fig.update_layout(legend_traceorder= 'reversed')
fig.show()

In [ ]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output

app_Graph1 = JupyterDash(__name__)

app_Graph1.layout = html.Div([dcc.Markdown(children = 'Seleziona la **data di inizio**'),    
                      dcc.Dropdown(id = 'dropdown_start_input',
                                   options = df3.Date.unique(),
                                   value = 'January  2017'),
                      dcc.Markdown(children = "---"),
                      dcc.Markdown(children = "Seleziona la **data di fine**"),
                      dcc.Dropdown(id = 'dropdown_end_input',
                                   options = df3.Date.unique(),
                                   value = 'December  2022'),
                      dcc.Markdown(children = '---'),
                      dcc.Graph(id = 'figure_output'),
                      dcc.Markdown(children = '---')],
                      # style = {'width': '20%'}
                      #use 100% of your brain! l'ultimo style si rifa a tutto l'HTML, di conseguenza 
                      #se vuoi mantenere 100% di un certo componente, devi moltiplicarlo per multipli di 100% 
                      
)

app_Graph1.run_server(mode = 'external')
@app_Graph1.callback(Output(component_id = 'figure_output', component_property = 'figure'),
              Input(component_id = 'dropdown_start_input', component_property = 'value'),
              Input(component_id = 'dropdown_end_input', component_property = 'value'))

def update_figure_output(start,end):
  df5 = df4
  pd.to_datetime(df5['Date'])
  df6 = df5.Date.between(start,end)
  df6 = df5.loc[df6]

  fig0 = px.line(df6, y='EV_Total', x='Date', color = 'ID_Colore_Totale', color_discrete_map= {'Totale' : 'red'})
  fig1 = px.bar(df6, y='Plugin_Hybrid_EV', x='Date', color = 'ID_Colore_Plugin', color_discrete_map= {'Ibrido plugin' : '#3366CC'})
  fig2 = px.bar(df6, y='Battery_EV', x='Date', color = 'ID_Colore_Batteria', color_discrete_map= {'Batteria' : "#0099c6"})
  fig = go.Figure(data = fig1.data + fig2.data + fig0.data)
  fig.update_layout(barmode="stack", clickmode="event+select", legend_traceorder= 'reversed', transition = {'duration': 1000})
  return fig

**GRAFICO 2**
---
Descrizione: 

In [ ]:
#DASH FUNZIONANTE CON County e Make DINAMICO -- NON ELIMINAREEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE


#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin
#vogliamo gestire il caso "non ci sono veicoli di questo Make nella County selezionata"? Sarebbe certamente più carino. Esempio: PORSCHE - Asotin

from jupyter_dash import JupyterDash
from decimal import Decimal, ROUND_DOWN
from dash import Dash, html, dcc, Input, Output

app_Graph2 = JupyterDash(__name__)

pop1 = pop
Makers = np.append(pop_tmt.Make.unique(), '-default- Tutti')
County = np.append(pop_tmt.County.unique(), '-default- Tutte (statale)')

app_Graph2.layout = html.Div([dcc.Markdown(children = 'Seleziona il **marchio**'),    
                      dcc.Dropdown(id = 'dropdown_marchio',
                                   options = sorted(Makers),
                                   value = '-default- Tutti'),
                      dcc.Markdown(children = "Seleziona la **contea**"),
                      dcc.Dropdown(id = 'dropdown_contea',
                                   options = sorted(County),
                                   value = '-default- Tutte (statale)'),
                      dcc.Markdown(children = '---'),
                      dcc.Graph(id = 'figure_output'),
                      dcc.Markdown(children = '---')],
                      # style = {'width': '20%'}
                      #use 100% of your brain! l'ultimo style si rifa a tutto l'HTML, di conseguenza 
                      #se vuoi mantenere 100% di un certo componente, devi moltiplicarlo per multipli di 100% 
                      
)

@app_Graph2.callback(Output(component_id = 'figure_output', component_property = 'figure'),
              Input(component_id = 'dropdown_marchio', component_property = 'value'),
              Input(component_id = 'dropdown_contea', component_property = 'value'))

def pix_pie (make, county):
  if (county == '-default- Tutte (statale)' and make == '-default- Tutti'): 
    test = pop_tmt
  elif(county == '-default- Tutte (statale)'): 
    test = pop_tmt[pop_tmt['Make'] == make]
  elif(make == '-default- Tutti'):
    test = pop_tmt[pop_tmt['County'] == county]
  else:
    test = pop_tmt[pop_tmt['County'] == county]
    test = test[test['Make'] == make]
  
  counts = test['Model'].value_counts().reset_index()
  counts.columns = ['Model', 'Number_of_Vehicles']
  total_count = counts['Number_of_Vehicles'].sum()
  counts['percent_of_total'] = counts['Number_of_Vehicles'] / total_count * 100
  counts['percent_of_total'] = counts['percent_of_total'].round(decimals = 2)
  
  #Rinomino tutti i modelli con percentuale minore di 2 con others, analogo al sito di Washington
  counts.loc[counts['percent_of_total'] < 2, 'Model'] = "Others"
  
  #Plotto la tortina
  fig = px.pie(counts, values = 'Number_of_Vehicles' , names = 'Model', hover_data=['percent_of_total'], labels={'percent_of_total':'percent of total','Number_of_Vehicles':'Number of Vehicles'})
  return fig

app_Graph2.run_server(mode = 'external')

**GRAFICO 3**
---
Descrizione: 

In [ ]:
import pandas as pd
import json
import requests
import plotly.express as px
import geopandas as gpd

# url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"

# response = requests.get(url) 
# data = json.loads(response.content)

# df = pd.json_normalize(data['features'])


# # filter the dataframe to include only the rows corresponding to the state of Washington
# df_wa = df[df['properties.STATE'] == '53']


# # URL for the raw JSON file on Github
url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

# # Make a GET request to the URL
response = requests.get(url)

# # Check if the request was successful
if response.status_code == 200:
    # Load the JSON data from the response
    data = json.loads(response.content)
    # Do something with the data...
else:
    # Handle the error
    print("Failed to download data. Status code:", response.status_code)

# Filter only the counties of Washington
filtered_data = {"type": "FeatureCollection", "features": []}
for feature in data["features"]:
    if feature["properties"]["STATE"] == "53":
        filtered_data["features"].append(feature)


tmp = adopt[adopt.Date == "July 31 2022"]

fig = px.choropleth(tmp[tmp.User == "Passenger" ], geojson = filtered_data, 
                    featureidkey= "properties.NAME" ,
                    locations= "County",
                    color = 'Percent_EV',
                    fitbounds = "locations",
                    hover_name = 'Vehicles_Total',

                    )

fig.show()




In [ ]:
from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output
import json
import requests
import plotly.express as px
import geopandas as gpd

app_Graph1 = JupyterDash(__name__)

app_Graph1.layout = html.Div([dcc.Markdown(children = 'Seleziona la **data**'),    
                      dcc.Dropdown(id = 'dropdown_start_input',
                                   options = adopt.Date.unique(),
                                   value = 'January  2017'),
                      dcc.Markdown(children = '---'),
                       dcc.Dropdown(id = 'dropdown_start_input1',
                                   options = ['Battery_EV' , 'Plugin_Hybrid_EV'],
                                   value = 'Battery_EV'),
                      dcc.Graph(id = 'figure_output'),
                      dcc.Markdown(children = '---')],
                      # style = {'width': '20%'}
                      #use 100% of your brain! l'ultimo style si rifa a tutto l'HTML, di conseguenza 
                      #se vuoi mantenere 100% di un certo componente, devi moltiplicarlo per multipli di 100% 
                      
)

@app_Graph1.callback(Output(component_id = 'figure_output', component_property = 'figure'),
              Input(component_id = 'dropdown_start_input', component_property = 'value'),
              Input(component_id = 'dropdown_start_input1', component_property = 'value'))
             

def update_figure_output(start,type):


 # # URL for the raw JSON file on Github
 url='https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'

 # # Make a GET request to the URL
 response = requests.get(url)

 # # Check if the request was successful
 if response.status_code == 200:
    # Load the JSON data from the response
    data = json.loads(response.content)
    # Do something with the data...
 else:
    # Handle the error
     print("Failed to download data. Status code:", response.status_code)

 # Filter only the counties of Washington
 filtered_data = {"type": "FeatureCollection", "features": []}
 for feature in data["features"]:
     if feature["properties"]["STATE"] == "53":
         filtered_data["features"].append(feature)


 tmp = adopt[adopt.Date == start]
 tmp1 = tmp[tmp.User == "Passenger"]
 median = tmp1[type].mean()
 fig = px.choropleth(tmp1, geojson = filtered_data, 
                    featureidkey= "properties.NAME" ,
                    locations= "County",
                    color = type ,
                    hover_name = 'Vehicles_Total',
                    range_color= (0,median)
                    
                    )
 fig = fig.update_geos(fitbounds = "locations", visible = True)
 return fig

 
app_Graph1.run_server(mode = 'external')





In [ ]:
adopt 